# Notebook 02: Processamento de Dados Médicos

Este notebook processa os dados médicos carregados no notebook anterior. Aqui vamos:

1. **Anonimizar** dados sensíveis (conformidade LGPD/HIPAA)
2. **Processar** cada entrada do dataset
3. **Limpar** e normalizar textos
4. **Dividir** textos em chunks otimizados
5. **Validar** qualidade dos dados processados

## 📋 Pré-requisitos

- Notebook 01 executado com sucesso (dataset carregado)
- Variáveis de ambiente configuradas
- Dependências instaladas

## 🔄 Ordem de Execução

Execute as células **sequencialmente** (de cima para baixo).


In [ ]:
# ============================================================================
# ETAPA 0: IMPORTAÇÃO DE BIBLIOTECAS E CONFIGURAÇÕES
# ============================================================================

import sys
from pathlib import Path

# Adiciona o diretório raiz ao path
root_dir = Path.cwd().parent
sys.path.insert(0, str(root_dir))

# Importa módulos do pipeline RAG
from scripts.data_loader import load_medical_dataset
from scripts.data_processor import (
    process_medical_entry,
    process_batch,
    filter_valid_entries
)
from scripts.text_splitter import MedicalTextSplitter, create_text_splitter
from config.settings import get_settings

# Carrega configurações
settings = get_settings()

print("✅ Bibliotecas importadas com sucesso!")


In [ ]:
# ============================================================================
# ETAPA 1: CARREGAMENTO DO DATASET (SE NÃO JÁ CARREGADO)
# ============================================================================
# Se você executou o notebook 01, pode pular esta célula
# Caso contrário, execute para carregar o dataset

# Descomente as linhas abaixo se necessário:
# data_path = settings.MEDICAL_DATA_PATH
# raw_data = load_medical_dataset(data_path)
# print(f"✅ Dataset carregado: {len(raw_data)} entradas")

# Se você já tem raw_data do notebook anterior, apenas confirme:
try:
    print(f"✅ Dataset já carregado: {len(raw_data)} entradas")
except NameError:
    print("⚠️  Execute a célula acima para carregar o dataset")
    raise


In [ ]:
# ============================================================================
# ETAPA 2: PROCESSAMENTO DE DADOS MÉDICOS
# ============================================================================
# Esta etapa processa cada entrada do dataset:
# - Combina múltiplos contextos em texto único
# - Aplica anonimização de dados sensíveis
# - Formata metadados estruturados
# - Prepara dados para embedding

print("=" * 80)
print("🔄 PROCESSANDO DADOS MÉDICOS")
print("=" * 80)
print(f"Total de entradas a processar: {len(raw_data)}")
print(f"Anonimização: Habilitada (conformidade LGPD/HIPAA)")
print("-" * 80)

# Processa todas as entradas em lote
processed_entries = process_batch(
    raw_data,
    anonymize=True,  # Habilita anonimização
    show_progress=True
)

print(f"\n✅ Processamento concluído!")
print(f"   Entradas processadas: {len(processed_entries)}")
print("=" * 80)


In [ ]:
# ============================================================================
# ETAPA 3: FILTRAGEM DE ENTRADAS VÁLIDAS
# ============================================================================
# Remove entradas muito curtas ou inválidas que não são adequadas
# para embedding e busca

print("=" * 80)
print("🔍 FILTRANDO ENTRADAS VÁLIDAS")
print("=" * 80)

total_before = len(processed_entries)
min_text_length = 50  # Tamanho mínimo em caracteres

valid_entries = filter_valid_entries(
    processed_entries,
    min_text_length=min_text_length
)

total_after = len(valid_entries)
removed = total_before - total_after

print(f"Entradas antes da filtragem: {total_before}")
print(f"Entradas após filtragem: {total_after}")
print(f"Entradas removidas: {removed} (muito curtas ou inválidas)")
print(f"Taxa de retenção: {total_after/total_before*100:.1f}%")
print("=" * 80)


In [ ]:
# ============================================================================
# ETAPA 4: DIVISÃO EM CHUNKS
# ============================================================================
# Divide textos longos em chunks menores para otimizar:
# - Embedding (chunks menores são mais eficientes)
# - Busca (recuperação mais precisa)
# - Armazenamento (melhor uso do Pinecone)

print("=" * 80)
print("✂️  DIVIDINDO TEXTOS EM CHUNKS")
print("=" * 80)
print(f"Chunk size: {settings.CHUNK_SIZE} caracteres")
print(f"Chunk overlap: {settings.CHUNK_OVERLAP} caracteres")
print("-" * 80)

# Cria divisor de texto
text_splitter = create_text_splitter(
    chunk_size=settings.CHUNK_SIZE,
    chunk_overlap=settings.CHUNK_OVERLAP
)

# Divide todas as entradas em chunks
all_chunks = text_splitter.split_batch(
    valid_entries,
    preserve_metadata=True,
    show_progress=True
)

print(f"\n✅ Divisão em chunks concluída!")
print(f"   Entradas originais: {len(valid_entries)}")
print(f"   Total de chunks gerados: {len(all_chunks)}")
print(f"   Média de chunks por entrada: {len(all_chunks)/len(valid_entries):.2f}")
print("=" * 80)


In [ ]:
# ============================================================================
# ETAPA 5: VISUALIZAÇÃO DE EXEMPLOS PROCESSADOS
# ============================================================================
# Exibe exemplos de chunks processados para verificação visual

print("=" * 80)
print("📄 EXEMPLOS DE CHUNKS PROCESSADOS")
print("=" * 80)

# Mostra alguns exemplos
for i, chunk in enumerate(all_chunks[:3], 1):
    print(f"\n{'='*80}")
    print(f"CHUNK {i}")
    print(f"{'='*80}")
    print(f"Article ID: {chunk['article_id']}")
    print(f"Chunk Index: {chunk['chunk_index']}")
    print(f"Tamanho: {len(chunk['text'])} caracteres")
    print(f"\nTexto (primeiros 300 caracteres):")
    print(f"{chunk['text'][:300]}...")
    print(f"\nMetadados:")
    for key, value in chunk['metadata'].items():
        if isinstance(value, str) and len(value) > 100:
            print(f"  {key}: {value[:100]}...")
        else:
            print(f"  {key}: {value}")

print(f"\n{'='*80}")
print("✅ Visualização concluída!")
print("=" * 80)


## ✅ Conclusão da Etapa 2

Neste notebook você:
- ✅ Processou todas as entradas do dataset
- ✅ Aplicou anonimização de dados sensíveis
- ✅ Filtrou entradas inválidas
- ✅ Dividiu textos em chunks otimizados
- ✅ Validou a qualidade dos dados processados

## 📌 Próximos Passos

Agora você está pronto para o próximo notebook:
- **Notebook 03**: Geração de embeddings e ingestão no Pinecone
- Conexão com Pinecone
- Geração de embeddings
- Ingestão em lotes
